In [1]:
from kth_dataset import KTH_Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from LSTMscratch import LSTMscratch
import torch
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
from Model_Wrapper import Wrapper


In [2]:
# Create a dataset and a DataLoader
transforms = transforms.Compose([
        transforms.ToTensor(),  # Convert image to tensor
        transforms.Normalize((0.5,), (0.5,)),  # Normalize to [-1, 1]
    ])
# Initialize dataset
dataset = KTH_Dataset(root_dir="/home/nfs/inf6/data/datasets/kth_actions/processed", sequence_length=15, transform=transforms,use_saved_samples=True)

In [3]:
dataset.__len__()


281330

In [4]:
train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=0.2)

In [6]:
datasets = {}

In [7]:
datasets['train'] = Subset(dataset, train_idx)
datasets['val'] = Subset(dataset, val_idx)

In [8]:
train_loader = DataLoader(datasets['train'], batch_size=64, shuffle=True)
test_loader = DataLoader(datasets['train'], batch_size=64, shuffle=False)
print(next(iter(test_loader))[0].size())

torch.Size([64, 15, 1, 64, 64])


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
model = LSTMscratch(input_dim=64, hidden_dim=16, number_of_layers=2, device=device)

In [27]:
# model_t = SequentialClassifierWithCells(emb_dim=128, hidden_dim=128, num_layers=2, mode="zeros")
model = model.to(device)


In [13]:
# classification loss function
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

# Decay LR by a factor of 0.1 every 5 epochs
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.2)

In [29]:
device

device(type='cuda')

In [15]:
model = model.to(device)

In [ ]:
# train_loss, val_loss, loss_iters, valid_acc = train_model(
#         model=model, optimizer=optimizer, scheduler=scheduler, criterion=criterion,
#         train_loader=train_loader, valid_loader=test_loader, num_epochs=10
#     )
classes = ["Boxing","HandClapping","HandWaving","Jogging","Running","Walking"]
train = Wrapper(model = model, device = device, criterion = criterion, optimizer = optimizer,writer=None,show_progress_bar= True)
train.train(2,train_loader,test_loader,classes)
train.valid_accuracy()
train.plot_loss_acc()

  0%|          | 0/3517 [00:00<?, ?it/s]

Epoch 1 Iter 201: loss 1.75443.:   6%|▌         | 203/3517 [03:40<58:29,  1.06s/it]  